## Кривоногов Н.В., PyTorch, практическое задание № 3

Будем практиковаться на датасете недвижимости (sklearn.datasets.fetch_california_housing)

Ваша задача:
1. Создать Dataset для загрузки данных
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import torch.nn.functional as F
import torch.nn as nn

import warnings
warnings.filterwarnings('ignore')

In [2]:
# загрузка датасета и разбиение на обучающую и тестовую выборку: 

X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [3]:
# создание класса Dataset: 

class MyOwnFCH(torch.utils.data.Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = torch.from_numpy(X_data).type(torch.float)
        self.y_data = torch.from_numpy(y_data).type(torch.float)
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [4]:
# создание классов нейросети: 

class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation

    def forward(self, x):
        x = self.fc(x)
        if self.activation == "relu":
            return F.relu(x)
        if self.activation == "sigmoid":
            return F.sigmoid(x)
        raise RuntimeError


class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.fc2 = Perceptron(hidden_dim, 1, "relu")

    def forward(self, x):
        x = self.fc1(x)
        x = self.dp(x)
        x = self.bn(x)
        x = self.fc2(x)
        return x

In [5]:
# создание экземпляров класса Dataset: 

train_dataset = MyOwnFCH(X_train, y_train)

test_dataset = MyOwnFCH(X_test, y_test)

In [6]:
# создание экземпляров класса DataLoader: 

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

In [7]:
# инициализация нейросети: 

net = FeedForward(8, 8)

net

FeedForward(
  (fc1): Perceptron(
    (fc): Linear(in_features=8, out_features=8, bias=True)
  )
  (dp): Dropout(p=0.25, inplace=False)
  (bn): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Perceptron(
    (fc): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [8]:
# функция потерь для задачи регрессии: 

criterion = nn.MSELoss()

In [9]:
# обучение нейросети: 

Adam_opt = torch.optim.Adam(net.parameters(), lr=0.0001)
RMSProp_opt = torch.optim.RMSprop(net.parameters(), lr=0.0001)
SGD_opt = torch.optim.SGD(net.parameters(), lr=0.0001)

optimizers = [Adam_opt, RMSProp_opt, SGD_opt]

for optimizer in optimizers:
    print('_________________________________')
    print('Оптимизатор: ', optimizer)
    print('Обучение: ')
    
    num_epochs = 20

    for epoch in range(num_epochs):
        running_loss, running_items, r2 = 0.0, 0.0, 0.0

        for i, data in enumerate(train_loader):
            inputs, labels = data[0], data[1]

            # обнуляю градиент: 
            optimizer.zero_grad()

            outputs = net(inputs)                # предсказание
            loss = criterion(outputs, labels)    # ошибка
            loss.backward()                      # градиенты
            optimizer.step()                     # шаг оптимизации

            running_loss += loss.item()
            running_items += len(labels)

            predict = outputs.data.numpy()
            train_target = labels.view(labels.shape[0], 1).numpy()
            r2 += r2_score(train_target, predict)

            # вывожу статистику о процессе обучения: 
            if i % 40 == 0:
                net.eval()

                data = list(test_loader)[0]

                test_outputs = net(data[0])
                test_predict = test_outputs.data.numpy()
                test_target = data[1].view(data[1].shape[0], 1)
                test_r2 = r2_score(test_target, test_predict)

                print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                      f'Step [{i + 1}/{len(train_loader)}]. ' \
                      f'Loss: {running_loss / running_items:.3f}. ' \
                      f'Train r2: {r2:.3f}. ' \
                      f'Test r2: {test_r2:.3f}')

                running_loss, running_items, r2 = 0.0, 0.0, 0.0

                net.train()

    print('Обучение закончено!')

_________________________________
Оптимизатор:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
Обучение: 
Epoch [1/20]. Step [1/121]. Loss: 0.033. Train r2: -2.179. Test r2: -4.036
Epoch [1/20]. Step [41/121]. Loss: 0.034. Train r2: -97.035. Test r2: -3.319
Epoch [1/20]. Step [81/121]. Loss: 0.034. Train r2: -93.575. Test r2: -3.341
Epoch [1/20]. Step [121/121]. Loss: 0.035. Train r2: -94.435. Test r2: -3.223
Epoch [2/20]. Step [1/121]. Loss: 0.030. Train r2: -2.322. Test r2: -3.235
Epoch [2/20]. Step [41/121]. Loss: 0.034. Train r2: -89.967. Test r2: -3.192
Epoch [2/20]. Step [81/121]. Loss: 0.035. Train r2: -91.631. Test r2: -3.187
Epoch [2/20]. Step [121/121]. Loss: 0.032. Train r2: -91.497. Test r2: -3.137
Epoch [3/20]. Step [1/121]. Loss: 0.039. Train r2: -2.472. Test r2: -3.124
Epoch [3/20]. Step [41/121]. L

Epoch [6/20]. Step [41/121]. Loss: 0.011. Train r2: -2.674. Test r2: -0.088
Epoch [6/20]. Step [81/121]. Loss: 0.011. Train r2: -3.032. Test r2: -0.141
Epoch [6/20]. Step [121/121]. Loss: 0.011. Train r2: -3.230. Test r2: -0.142
Epoch [7/20]. Step [1/121]. Loss: 0.009. Train r2: -0.090. Test r2: -0.141
Epoch [7/20]. Step [41/121]. Loss: 0.011. Train r2: -2.851. Test r2: -0.154
Epoch [7/20]. Step [81/121]. Loss: 0.011. Train r2: -2.412. Test r2: -0.167
Epoch [7/20]. Step [121/121]. Loss: 0.011. Train r2: -2.045. Test r2: -0.169
Epoch [8/20]. Step [1/121]. Loss: 0.009. Train r2: -0.030. Test r2: -0.171
Epoch [8/20]. Step [41/121]. Loss: 0.011. Train r2: -2.214. Test r2: -0.190
Epoch [8/20]. Step [81/121]. Loss: 0.011. Train r2: -2.093. Test r2: -0.178
Epoch [8/20]. Step [121/121]. Loss: 0.011. Train r2: -1.617. Test r2: -0.225
Epoch [9/20]. Step [1/121]. Loss: 0.010. Train r2: -0.010. Test r2: -0.231
Epoch [9/20]. Step [41/121]. Loss: 0.011. Train r2: -1.828. Test r2: -0.249
Epoch [9/20]

Epoch [12/20]. Step [81/121]. Loss: 0.011. Train r2: -0.256. Test r2: -0.312
Epoch [12/20]. Step [121/121]. Loss: 0.010. Train r2: -0.408. Test r2: -0.332
Epoch [13/20]. Step [1/121]. Loss: 0.009. Train r2: 0.008. Test r2: -0.339
Epoch [13/20]. Step [41/121]. Loss: 0.011. Train r2: -0.230. Test r2: -0.316
Epoch [13/20]. Step [81/121]. Loss: 0.010. Train r2: -0.405. Test r2: -0.336
Epoch [13/20]. Step [121/121]. Loss: 0.010. Train r2: -0.254. Test r2: -0.321
Epoch [14/20]. Step [1/121]. Loss: 0.011. Train r2: -0.000. Test r2: -0.320
Epoch [14/20]. Step [41/121]. Loss: 0.010. Train r2: -0.402. Test r2: -0.329
Epoch [14/20]. Step [81/121]. Loss: 0.010. Train r2: -0.245. Test r2: -0.314
Epoch [14/20]. Step [121/121]. Loss: 0.010. Train r2: -0.411. Test r2: -0.316
Epoch [15/20]. Step [1/121]. Loss: 0.009. Train r2: -0.002. Test r2: -0.316
Epoch [15/20]. Step [41/121]. Loss: 0.011. Train r2: -0.262. Test r2: -0.346
Epoch [15/20]. Step [81/121]. Loss: 0.010. Train r2: -0.384. Test r2: -0.325


Нейросеть с оптимизатором Adam показала более высокие результаты. 